In [4]:
import pandas as pd
import numpy as np
import IntegratedGradients as ig
import tensorflow as tf

def one_hot_encode(df, col='utr', seq_len=50):
    nuc_d = {'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1], 'n':[0,0,0,0]}
    vectors=np.empty([len(df),seq_len,4])
    
    for i,seq in enumerate(df[col].str[:seq_len]): 
        seq = seq.lower()
        a = np.array([nuc_d[x] for x in seq])
        vectors[i] = a
    return vectors

In [2]:
data = pd.read_csv('/Users/frederickkorbel/Documents/projects/paper/data/MRL_pred.csv', index_col = False, nrows = 100)
mrl_model = tf.keras.models.load_model('/Users/frederickkorbel/Documents/projects/paper/data/seelig/Supplementary Code/modeling/saved_models/main_MRL_model.hdf5')

seq = one_hot_encode(data, seq_len = 50)
ind = data.index

igres = ig.integrated_gradients(mrl_model)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
